# Demo:  Computing Gradients

Most numerical optimization methods require that we compute gradients of the loss function that we are attempting to minimize.  In this demo, we illustrate how to compute gradients efficiently in python for a few simple examples.  As much as possible, we avoid for loops for fast implementation.

In [2]:
import numpy as np

## Example 1:  A Simple Vector-Input Function

Suppose `f(w) = w_0^2 + 2w_0w_1^3`.  Then the function below computes `f(w)` its gradient.

In [3]:
def feval(w):

    # Function
    f = w[0]**2 + 2*w[0]*(w[1]**3)

    # Gradient
    df0 = 2*w[0]+2*(w[1]**3)
    df1 = 6*w[0]*(w[1]**2)
    fgrad = np.array([df0, df1])
    
    return f, fgrad

# Point to evaluate 
w = np.array([2,4])
f, fgrad = feval(w)

print('f     = %f' % f)
print('fgrad = ' + str(fgrad))


f     = 260.000000
fgrad = [132 192]


## Example 2:  Non-Linear Least Squares for an Exponential Model

Consider an exponential model 

    yhat = a*exp(-b*x)
    
for parameters `w=[a,b]`.  Given training data `(x[i],y[i])` a natural loss function is given by

    J(w) := \sum_i (y[i] - yhat[i])**2,   yhat[i] = a*exp(-b*x[i])
    
The following code computes the the loss function `J(w)` and its gradient `dJ/dw`.

In [4]:
def Jeval(w):
    
    # Unpack vector
    a = w[0]
    b = w[1]
    
    # Compute the loss function
    yerr = y-a*np.exp(-b*x)
    J = 0.5*np.sum(yerr**2)

    # Compute the gradient
    dJ_da = -np.sum( yerr*np.exp(-b*x))
    dJ_db = np.sum( yerr*a*x*np.exp(-b*x))
    Jgrad = np.array([dJ_da, dJ_db])
    return J, Jgrad

# Generate some random data
ny = 100
y = np.random.randn(ny)
x = np.random.rand(ny)

# Some arbitrary parameters 
# to compute the gradient at
a = 1
b = 2
w = np.array([a,b])

J, Jgrad = Jeval(w)
print('Jgrad = ' + str(Jgrad))


Jgrad = [27.4427924  -6.12391384]


## Example 3:  Loss Function for a Log-Linear Model

Now suppose we have a logarithmic linear model:

    yhat = log(z),  z = w_0 + \sum_j w_jx_j
    
If we have data `X, y`, the prediction on sample `i` is:

    yhat_i = log(z_i),  z_i = w_0 + \sum_j w_j*X_{ij}
 
    
Suppose we use MSE loss:

    J(w) = \sum_i (y_i - yhat_i )**2
    
To compute the components `dJ/dw_j`, first write `z = Aw` where `A=[1 X]`, the matrix with ones on the first column.
Then, `dz_i/dw_j = A_{ij}`.  Also, `dyhat_i / dz_i = 1/z_i`, so with the multi-variable chain rule: 

    dJ/dw_j = \sum_i dJ/dyhat_i * dyhat_i / dw_j 
            = \sum_i (dJ/dyhat_i) * (dyhat_i / dz_i) * (dz_i / dw_j) = 
            = 2*\sum_i (yhat_i - y_i) * 1/z_i* A_{ij}
    
We can implement the loss and gradient computation as follows:


In [5]:
def Jeval(w,X,y):

    # Create matrix A=[1 X]
    n = X.shape[0]
    A = np.column_stack((np.ones(n), X))

    # Compute function
    z = A.dot(w)
    yhat = np.log(z)
    J = np.sum((y-yhat)**2)
    
    # Compute gradient
    dJ_dz = 2*(yhat-y)/z
    Jgrad = A.T.dot(dJ_dz)
    
    return J, Jgrad  

Whenever you implement a gradient, you should always check the gradients.  Errors in the gradient are the number 

*  Take two points `w0` and `w1` close to one another
*  Verify that `J(w1)-J(w0)` is close to `Jgrad(w0).dot(w1-w0)`. 

In [6]:
# Generate random positive data
n = 100
d = 5
X = np.random.uniform(0,1,(n,d))
w0 = np.random.uniform(0,1,(d+1,))
y = np.random.uniform(0,2,(n,))

# Compute function and gradient at point w0
J0, Jgrad0 = Jeval(w0,X,y)

# Take a small perturbation
step = 1e-4
w1 = w0 + step*np.random.normal(0,1,(d+1,))

# Evaluate the function at perturbed point
J1, Jgrad1 = Jeval(w1,X,y)

dJ = J1-J0
dJ_est = Jgrad0.dot(w1-w0)
print('Actual difference:     %12.4e' % dJ)
print('Estimated difference:  %12.4e' % dJ_est)

Actual difference:       2.9405e-03
Estimated difference:    2.9398e-03


## Example 3:  A Function of a Matrix.

Suppose `f(W) = a'*W*b`.  Then, `fgrad(W) = a*b.T`.

In [7]:
def feval(W,a,b):
    # Function
    f = a.dot(W.dot(b))

    # Gradient -- Use python broadcasting
    fgrad = a[:,None]*b[None,:]
    
    return f, fgrad
    
# Some random data
m = 4
n = 3
W = np.random.randn(m,n)
a = np.random.randn(m)
b = np.random.randn(n)

f, fgrad = feval(W,a,b)





## In-Class Exercise:  An Exponential Model

Consider a model,

    yhat = w[0]*exp(-w[1]*(x-w[2])**2/2)
   
where the parameter `w[2] > 0` is positive.

Now, suppose that, given data `x` and `y`, we want to minimize the MSE loss function,

    J = mean( (y[i] - yhat[i])**2 ) 
   
Complete the following function to compute `J` and its gradient for parameters `w` and data (`x,y`).

In [8]:
def Jeval(w,X,y):
    # TODO    
    return J, Jgrad

Test the gradient:

* Generate random `x` and `y` vectors with length `n=100`.
* Compute `J0, Jgrad0`, the loss function and its gradient, at the `w0` value below
* Compute some `w1`close to `w0`. 
* Compute `J1, Jgrad1`, the loss function and its gradient, at `w1`.
* Check that `J1-J0` is approximately `Jgrad0.dot(w1-w0)`.


In [9]:
w0 = np.array([0.4, 2, 2])

In [10]:
a = np.array([1,1])
b = np.array([1,1])
x = np.array([1,2,3,4])
y = np.array([1,2,3,4])


In [12]:
np.sum(a[None,:]*np.exp(-(x[:,None]-b[None,:])**2/2), axis=1)

array([2.        , 1.21306132, 0.27067057, 0.02221799])

In [13]:
np.exp(-(x[:,None]-b[None,:])**2/2).dot(a)

array([2.        , 1.21306132, 0.27067057, 0.02221799])